## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [43]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [44]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [45]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [46]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = [
    'T9',
    'T13',
    'T12',
    'T2',
    'T25',
    'T14',
    'T29',
    'T15',
    'T17'
]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [47]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [48]:
# Look at CLASSIFICATION value counts >1
vcs = application_df['CLASSIFICATION'].value_counts()
vcs[vcs>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [49]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications = application_df['CLASSIFICATION'].unique()
classifications_to_replace = [c for c in classifications if c not in ['C1000', 'C1200', 'C2000', 'C2100', 'C3000', 'C7000', 'C1700']]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [80]:
# Remove status column
application_df = application_df.drop(columns=['STATUS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,<25000,N,5000,1
1,T3,Independent,C2000,Preservation,Other,<25000,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,<25000,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,<25000,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,25000+,N,142590,1


In [81]:
# Remove special considerations column
application_df = application_df.drop(columns='SPECIAL_CONSIDERATIONS')
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,<25000,5000,1
1,T3,Independent,C2000,Preservation,Other,<25000,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,<25000,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,<25000,6692,1
4,T3,Independent,C1000,Heathcare,Trust,25000+,142590,1


In [50]:
# Create a list of affliation values to be binned as 'other'
affiliations_to_replace = ['Family/Parent', 'National', 'Regional', 'Other']
for aff in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [51]:
# Create a list of organization values to be binned as 'other'
organizations_to_replace = ['Co-operative', 'Corporation']
for org in organizations_to_replace:
  application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org, 'Other')
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [70]:
# Bin income amounts
income_amts = application_df['INCOME_AMT'].unique()
low_income = ['0', '1-9999', '10000-24999']
high_income = [i for i in income_amts if i not in low_income]

for inc in low_income:
  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc, '<25000')

for inc in high_income:
  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc, '25000+')

application_df['INCOME_AMT'].value_counts()

<25000    25659
25000+     8640
Name: INCOME_AMT, dtype: int64

In [82]:
# Convert categorical data to numeric with `pd.get_dummies`
num_df = pd.get_dummies(application_df)
num_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_25000+,INCOME_AMT_<25000
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,1
3,6692,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
4,142590,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [83]:
# Split our preprocessed data into our features and target arrays
X = num_df.drop(columns='IS_SUCCESSFUL').values
y = num_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [94]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_dim = len(num_df.columns) - 1

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu', input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu', input_dim =input_dim))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=32, activation='tanh', input_dim =input_dim))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 16)                512       
                                                                 
 dense_46 (Dense)            (None, 16)                272       
                                                                 
 dense_47 (Dense)            (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [95]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [96]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5738 - accuracy: 0.7147
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7233
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7234
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7250
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5530 - accuracy: 0.7262
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7268
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7260
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7273
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7282
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7268

In [97]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5529 - accuracy: 0.7300 - 662ms/epoch - 2ms/step
Loss: 0.5529308915138245, Accuracy: 0.7300291657447815


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')